# VLBA conversion guide

In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

XRADIO version 0.0.35 already installed.


## Download dataset

Original VLBA dataset gotten from https://casaguides.nrao.edu/index.php?title=VLBA_Basic_Phase-referencing_Calibration_and_Imaging.

Reduced the data using:

```Python
mstransform(vis='VLBA_TL016B.ms/',outputvis='VLBA_TL016B_split_lsrk.ms',spw='0:0~5,1:0~5',field='0,1', timerange='2022/02/21/06:14:00~2022/02/21/07:55:45.00', regridms=True,outframe='lsrk',datacolumn='all')
```




In [2]:
import graphviper

graphviper.utils.data.download(file="VLBA_TL016B_split.ms")

[2024-08-22 15:52:04,338]  WARNING  graphviper:  File exists: /home/fedemp/ws_xradio_pointing/venv_xradio_python_39/lib/python3.9/site-packages/graphviper/utils/data/.dropbox 
[2024-08-22 15:52:04,338]     INFO  graphviper:  Updating file metadata information ...  
 

 Download List         
 ────────────────────── 
  VLBA_TL016B_split.ms

[2024-08-22 15:52:05,834]     INFO  graphviper:  File exists: VLBA_TL016B_split.ms 


## Convert to Processing Set

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

ms_file = "VLBA_TL016B_split.ms"

main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "VLBA_TL016B_split_lsrk.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

[2024-08-22 15:52:07,785]     INFO  graphviper:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBSERVATION_ID', 'FIELD_ID'] 
[2024-08-22 15:52:07,804]     INFO  graphviper:  Number of partitions: 4 
[2024-08-22 15:52:07,805]     INFO  graphviper:  OBSERVATION_ID [0], DDI [0], STATE [-1], FIELD [0], SCAN [0] 
[2024-08-22 15:52:08,108]  WARNING  graphviper:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-08-22 15:52:08,253]     INFO  graphviper:  OBSERVATION_ID [0], DDI [0], STATE [-1], FIELD [1], SCAN [0] 
[2024-08-22 15:52:08,570]  WARNING  graphviper:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-08-22 15:52:08,806]     INFO  graphviper:  OBSERVATION_ID [0], DDI [1], STATE [-1], FIELD [0], SCAN [0] 
[2024-08-22 15:52:09,092]  WARNING  graphviper:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-08-22 15:52:09,237]     INFO  graphviper:  OBSERV

## Processing Set

In [4]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_store=outfile)
ps.summary()

,name,obs_mode,shape,polarization,spw_name,field_name,source_name,field_coords,start_frequency,end_frequency
0,VLBA_TL016B_split_lsrk_2,obs_0,"(200, 55, 6, 2)","[RR, LL]",spw_1,[4C39.25_0],[Unknown],"[fk5, 9h27m03.01s, 39d02m20.85s]",5.068000e+09,5.070500e+09
1,VLBA_TL016B_split_lsrk_3,obs_0,"(540, 55, 6, 2)","[RR, LL]",spw_1,[J1154+6022_1],[Unknown],"[fk5, 11h54m04.54s, 60d22m20.82s]",5.068000e+09,5.070500e+09
2,VLBA_TL016B_split_lsrk_0,obs_0,"(200, 55, 6, 2)","[RR, LL]",spw_0,[4C39.25_0],[Unknown],"[fk5, 9h27m03.01s, 39d02m20.85s]",5.004000e+09,5.006500e+09
3,VLBA_TL016B_split_lsrk_1,obs_0,"(540, 55, 6, 2)","[RR, LL]",spw_0,[J1154+6022_1],[Unknown],"[fk5, 11h54m04.54s, 60d22m20.82s]",5.004000e+09,5.006500e+09


In [5]:
ps.keys()

dict_keys(['VLBA_TL016B_split_lsrk_2', 'VLBA_TL016B_split_lsrk_3', 'VLBA_TL016B_split_lsrk_0', 'VLBA_TL016B_split_lsrk_1'])

In [6]:
ps["VLBA_TL016B_split_lsrk_3"]

<xarray.Dataset> Size: 6MB
Dimensions:                     (time: 540, baseline_id: 55, frequency: 6,
                                 polarization: 2, uvw_label: 3)
Coordinates:
    antenna_id                  (baseline_id) int32 220B dask.array<chunksize=(55,), meta=np.ndarray>
    baseline_antenna1_name      (baseline_id) <U2 440B dask.array<chunksize=(55,), meta=np.ndarray>
    baseline_antenna2_name      (baseline_id) <U2 440B dask.array<chunksize=(55,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 440B 0 1 2 3 ... 51 52 53 54
  * frequency                   (frequency) float64 48B 5.068e+09 ... 5.07e+09
    mount                       (baseline_id) <U6 1kB dask.array<chunksize=(55,), meta=np.ndarray>
    name                        (baseline_id) <U2 440B dask.array<chunksize=(55,), meta=np.ndarray>
  * polarization                (polarization) <U2 16B 'RR' 'LL'
    station                     (baseline_id) <U2 440B dask.array<chunksize=(55,), meta=np.ndarray>
  * time                        (time) float64 4kB 1.645e+09 ... 1.645e+09
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 238kB dask.array<chunksize=(20, 55), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool 356kB dask.array<chunksize=(20, 55, 1, 2), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 238kB dask.array<chunksize=(20, 55), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 713kB dask.array<chunksize=(20, 55, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 3MB dask.array<chunksize=(20, 55, 1, 2), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 1MB dask.array<chunksize=(20, 55, 1, 2), meta=np.ndarray>
Attributes:
    data_groups:     {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'V...
    partition_info:  {'field_name': ['J1154+6022_1'], 'num_lines': 0, 'obs_mo...
    type:            visibility
    weather_xds:     <xarray.Dataset> Size: 486kB\nDimensions:         (stati...
    antenna_xds:     <xarray.Dataset> Size: 521kB\nDimensions:               ...

In [7]:
ps["VLBA_TL016B_split_lsrk_3"].attrs["antenna_xds"]

<xarray.Dataset> Size: 521kB
Dimensions:                   (name: 10, cartesian_pos_label: 3,
                               receptor_name: 2, sky_dir_label: 2,
                               gain_curve_time: 1, poly_term: 1,
                               antenna_id: 10, phase_cal_time: 642,
                               tone_label: 2)
Coordinates:
  * cartesian_pos_label       (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
  * gain_curve_time           (gain_curve_time) float64 8B 5.152e+09
    mount                     (name) <U6 240B dask.array<chunksize=(10,), meta=np.ndarray>
  * name                      (name) <U2 80B 'BR' 'FD' 'HN' ... 'OV' 'PT' 'SC'
  * phase_cal_time            (phase_cal_time) float64 5kB 5.152e+09 ... 5.15...
  * receptor_name             (receptor_name) <U21 168B '0' '1'
  * sky_dir_label             (sky_dir_label) <U3 24B 'ra' 'dec'
    station                   (name) <U2 80B dask.array<chunksize=(10,), meta=np.ndarray>
Dimensions without coordinates: poly_term, antenna_id, tone_label
Data variables: (12/13)
    ANTENNA_DISH_DIAMETER     (name) float64 80B dask.array<chunksize=(10,), meta=np.ndarray>
    ANTENNA_FEED_OFFSET       (name, cartesian_pos_label) float64 240B dask.array<chunksize=(10, 3), meta=np.ndarray>
    ANTENNA_POSITION          (name, cartesian_pos_label) float64 240B dask.array<chunksize=(10, 3), meta=np.ndarray>
    BEAM_OFFSET               (name, receptor_name, sky_dir_label) float64 320B dask.array<chunksize=(10, 2, 2), meta=np.ndarray>
    GAIN_CURVE                (name, gain_curve_time, poly_term, receptor_name) float32 80B dask.array<chunksize=(10, 1, 1, 2), meta=np.ndarray>
    GAIN_CURVE_INTERVAL       (name, gain_curve_time) float64 80B dask.array<chunksize=(10, 1), meta=np.ndarray>
    ...                        ...
    PHASE_CAL                 (antenna_id, phase_cal_time, receptor_name, tone_label) complex64 205kB dask.array<chunksize=(10, 642, 2, 2), meta=np.ndarray>
    PHASE_CAL_CABLE_CAL       (antenna_id, phase_cal_time) float64 51kB dask.array<chunksize=(10, 642), meta=np.ndarray>
    PHASE_CAL_INTERVAL        (antenna_id, phase_cal_time) float64 51kB dask.array<chunksize=(10, 642), meta=np.ndarray>
    PHASE_CAL_TONE_FREQUENCY  (antenna_id, phase_cal_time, receptor_name, tone_label) float64 205kB dask.array<chunksize=(10, 642, 2, 2), meta=np.ndarray>
    POLARIZATION_TYPE         (name, receptor_name) <U1 80B dask.array<chunksize=(10, 2), meta=np.ndarray>
    RECEPTOR_ANGLE            (name, receptor_name) float64 160B dask.array<chunksize=(10, 2), meta=np.ndarray>
Attributes:
    overall_telescope_name:  VLBA
    relocatable_antennas:    False